# 3. Treinando e Testando o Modelo

Após o pré-processamento completo dos dados, o próximo passo é construir um modelo de classificação para prever o **sucesso das startups**.

Inicialmente, testamos o **XGBClassifier**, que apresentou **ótimos resultados** em termos de acurácia e desempenho geral.  
No entanto, como nosso objetivo era utilizar **apenas modelos da biblioteca `scikit-learn`**, não pudemos seguir com o XGBoost.

Diante disso, optamos pelo **RandomForestClassifier**, que possui características semelhantes ao XGBClassifier:

- Baseado em **árvores de decisão**, mas construindo múltiplas árvores para formar um **ensemble robusto**.
- Capaz de lidar bem com **dados mistos** (numéricos e categóricos codificados) e **features com diferentes escalas**.
- Robusto contra **overfitting** quando comparado a uma única árvore de decisão.
- Muito eficiente para conjuntos de dados estruturados como o nosso, permitindo boa **generalização**.

Dessa forma, o RandomForest se tornou o modelo ideal para continuar os experimentos e análises preditivas neste dataset.


### 3.1 Importando o que usaremos

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np


### 3.2 Carregando os Dados Processados

Após o pré-processamento e feature engineering, salvamos os datasets tratados em arquivos CSV separados (`train_processado.csv` e `test_processado.csv`). 

In [12]:
df_train = pd.read_csv('train_processado.csv')
df_test = pd.read_csv('test_processado.csv')

### 3.3 Treinamento do Modelo: Random Forest

Nesta etapa, realizamos o treinamento do modelo de classificação para prever a variável `labels`. Inicialmente, testamos o `XGBClassifier`, que teve ótimo desempenho, mas como ele **não pertence à biblioteca `sklearn`**, optamos por usar o `RandomForestClassifier`, que possui mecânica semelhante e também é muito eficaz para datasets estruturados como este.


#### 3.3.1 Separação entre Features e Target

Nesta etapa, definimos as features (`X`) e a variável alvo (`y`). Também removemos colunas irrelevantes, como `id` e `situacao_funding`, que não fornecem informação útil para o modelo.

```python
# Remover colunas irrelevantes ou auxiliares
drop_cols = [col for col in ['id', 'situacao_funding'] if col in df_train.columns]

# Features (X) e Target (y)
X = df_train.drop(columns=['labels'] + drop_cols)
y = df_train['labels']
```
#### 3.3.2. Divisão em treino e validação

Aqui, dividimos os dados em conjuntos de treino e validação. Usamos 20% para validação e mantemos a proporção das classes com `stratify=y`.

```python
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,       # 20% dos dados para validação
    random_state=42,     # garante reprodutibilidade
    stratify=y           # mantém proporção das classes
)
```
#### 3.3.3. Definição do Modelo Base e Espaço de Hiperparâmetros

Definimos o modelo `RandomForestClassifier` base e o espaço de hiperparâmetros que será explorado no `RandomizedSearchCV`.

```python
from sklearn.ensemble import RandomForestClassifier

# Modelo base
rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # importante para lidar com desbalanceamento
)

# Espaço de hiperparâmetros
param_dist = {
    'n_estimators': [200, 500, 800, 1000],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
}
```

#### 3.3.4. Randomized Search para Otimização de Hiperparâmetros

Nesta etapa, usamos o `RandomizedSearchCV` para explorar combinações aleatórias dos hiperparâmetros definidos, buscando a melhor configuração para o modelo Random Forest. Essa abordagem é mais rápida do que testar todas as combinações possíveis (Grid Search) e ainda consegue encontrar bons resultados.

```python
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_dist,
    n_iter=50,          # número de combinações aleatórias testadas
    cv=3,               # validação cruzada com 3 folds
    scoring='accuracy', # métrica utilizada para avaliar desempenho
    n_jobs=-1,          # paraleliza o processo
    verbose=2,          # exibe o progresso
    random_state=42
)

# Treinar o modelo
random_search.fit(X_train, y_train)

# Exibir melhores parâmetros encontrados
print("Melhores parâmetros encontrados:", random_search.best_params_)

# Selecionar o melhor modelo final
best_rf = random_search.best_estimator_
```
#### 3.3.5. Avaliação do Modelo no Conjunto de Validação

Após treinar o modelo com os melhores hiperparâmetros encontrados pelo `RandomizedSearchCV`, é hora de avaliar seu desempenho no conjunto de validação. Nesta etapa, verificamos métricas essenciais para entender como o modelo generaliza para dados que ele ainda não viu.

```python
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Previsões no conjunto de validação
y_pred = best_rf.predict(X_val)

# 1. Acurácia
acc = accuracy_score(y_val, y_pred)
print(f"Acurácia no conjunto de validação: {acc:.4f}")

# 2. Relatório detalhado por classe
print("Relatório de Classificação:\n", classification_report(y_val, y_pred))

# 3. Matriz de Confusão
cm = confusion_matrix(y_val, y_pred)
print("Matriz de Confusão:\n", cm)
```

In [13]:

# 1. Separar features e target
drop_cols = [col for col in ['id', 'situacao_funding'] if col in df_train.columns]
X = df_train.drop(columns=['labels'] + drop_cols)
y = df_train['labels']

# 2. Dividir treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# Modelo base
rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # importante para dataset desbalanceado
)

# Espaço de hiperparâmetros mais amplo
param_dist = {
    'n_estimators': [200, 500, 800, 1000],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', None]
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_dist,
    n_iter=50,              # número de combinações aleatórias testadas
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Treinar
random_search.fit(X_train, y_train)

print("Melhores parâmetros encontrados:", random_search.best_params_)

# Melhor modelo
best_rf = random_search.best_estimator_


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END max_depth=40, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=15, n_estimators=500; total time=   0.9s
[CV] END max_depth=40, max_features=log2, min_samples_leaf=6, min_samples_split=15, n_estimators=500; total time=   0.9s
[CV] END max_depth=40, max_features=log2, min_samples_leaf=6, min_samples_split=15, n_estimators=500; total time=   0.9s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=15, n_estimators=500; total time=   0.9s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimator

## 3.4 Treinando o modelo com base nos melhores parâmetros passados.

### 3.4.1. Separação entre Features e Target

Primeiro, definimos as variáveis de entrada (`X`) e a variável alvo (`y`). Colunas que não trazem informação relevante, como `id` e `situacao_funding`, são removidas.

```python
drop_cols = [col for col in ['id', 'situacao_funding'] if col in df_train.columns]
X = df_train.drop(columns=['labels'] + drop_cols)
y = df_train['labels']
```
### 3.4.2. Divisão entre Treino e Validação

Nesta etapa, separamos os dados em conjuntos de treino e validação. O conjunto de treino será utilizado para ajustar o modelo, enquanto o conjunto de validação servirá para avaliar a performance do modelo em dados que ele ainda não viu. 

Usamos o parâmetro `test_size=0.2` para destinar 20% dos dados para validação. O parâmetro `stratify=y` garante que a proporção das classes no conjunto de treino e validação se mantenha igual à do dataset original, evitando desbalanceamento.

```python
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
```
### 3.4.3. Instanciação do Modelo

Nesta etapa, instanciamos o modelo `RandomForestClassifier` com os melhores hiperparâmetros encontrados na busca aleatória (RandomizedSearchCV).  

- `n_estimators=200`: número de árvores na floresta.  
- `max_depth=30`: profundidade máxima de cada árvore.  
- `min_samples_split=2`: número mínimo de amostras necessárias para dividir um nó.  
- `min_samples_leaf=1`: número mínimo de amostras em um nó folha.  
- `max_features='sqrt'`: número máximo de features a serem consideradas para a melhor divisão.  
- `class_weight='balanced'`: ajusta automaticamente pesos das classes para lidar com desbalanceamento.  
- `random_state=42`: garante reprodutibilidade dos resultados.  
- `n_jobs=-1`: utiliza todos os núcleos de CPU disponíveis para acelerar o treinamento.  

```python
best_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
```

### 3.4.4. Treinamento do Modelo

Nesta etapa, treinamos o modelo `RandomForestClassifier` utilizando os dados de treino (`X_train` e `y_train`).  
O modelo irá aprender padrões presentes nas features para prever a variável alvo `labels`.

```python
# Treinar o modelo
best_rf.fit(X_train, y_train)
```

### 3.4.5. Avaliação do Modelo

Predizemos os valores no conjunto de validação (`X_val`) e avaliamos a performance do modelo utilizando métricas importantes:

- **Acurácia:** proporção de previsões corretas.
- **Relatório de Classificação:** inclui precision, recall e F1-score por classe.
- **Matriz de Confusão:** mostra acertos e erros detalhados por classe.

```python
# Avaliação no conjunto de validação
y_pred = best_rf.predict(X_val)
print("Acurácia:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
print("Matriz de confusão:\n", confusion_matrix(y_val, y_pred))
```


In [14]:
# Separar features e target
drop_cols = [col for col in ['id', 'situacao_funding'] if col in df_train.columns]
X = df_train.drop(columns=['labels'] + drop_cols)
y = df_train['labels']

# Dividir treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Instanciar o modelo com os melhores parâmetros
best_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # importante para classes desbalanceadas
)

# Treinar o modelo
best_rf.fit(X_train, y_train)

# Avaliação no conjunto de validação
y_pred = best_rf.predict(X_val)
print("Acurácia:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
print("Matriz de confusão:\n", confusion_matrix(y_val, y_pred))


Acurácia: 0.7769230769230769
              precision    recall  f1-score   support

           0       0.73      0.59      0.65        46
           1       0.80      0.88      0.84        84

    accuracy                           0.78       130
   macro avg       0.76      0.73      0.74       130
weighted avg       0.77      0.78      0.77       130

Matriz de confusão:
 [[27 19]
 [10 74]]


### 3.5 Análise dos Resultados do Modelo

Após treinar o Random Forest, avaliamos seu desempenho no conjunto de validação. Os resultados obtidos foram:

- **Acurácia:** 77,7% das previsões estão corretas, o que indica um bom desempenho geral do modelo.
- **Precision e Recall por Classe:**
  - Classe `0` (startups que não atingiram o sucesso):
    - Precision: 0.73 → Das previsões feitas como classe 0, 73% estavam corretas.
    - Recall: 0.59 → Apenas 59% das startups realmente da classe 0 foram corretamente identificadas.
    - F1-Score: 0.65 → Métrica balanceada entre precision e recall.
  - Classe `1` (startups bem-sucedidas):
    - Precision: 0.80 → Das previsões feitas como classe 1, 80% estavam corretas.
    - Recall: 0.88 → 88% das startups realmente bem-sucedidas foram corretamente identificadas.
    - F1-Score: 0.84 → Excelente equilíbrio entre precision e recall.

- **Matriz de Confusão:**
[[27 19]
[10 74]]

  - Linha 0: 27 startups da classe 0 foram corretamente classificadas, 19 foram classificadas incorretamente como classe 1.
  - Linha 1: 74 startups da classe 1 foram corretamente classificadas, 10 foram classificadas incorretamente como classe 0.

**Conclusão:**  
O modelo apresenta desempenho melhor para a classe 1 (startups bem-sucedidas), conseguindo identificar a maioria corretamente. Para a classe 0, o recall é menor, indicando que algumas startups sem sucesso estão sendo classificadas como bem-sucedidas.  
Ainda assim, considerando o desbalanceamento do dataset, o resultado geral é muito bom e consistente com expectativas para este tipo de problema.


## 3.6 Previsão e Geração do Arquivo de Submissão

Nesta etapa, utilizamos o modelo Random Forest treinado para prever os rótulos do conjunto de teste e gerar o arquivo de submissão.

#### 3.6.1 Preparação das Features do Conjunto de Teste

Removemos colunas irrelevantes ou auxiliares (`id`, `situacao_funding` e `labels`, se existirem) para manter apenas as features que o modelo espera.

```python
drop_cols_test = [col for col in ['id', 'situacao_funding', 'labels'] if col in df_test.columns]
X_test = df_test.drop(columns=drop_cols_test)
```
#### 3.6.2. Previsão dos Rótulos

Nesta etapa, aplicamos o modelo Random Forest previamente treinado (`best_rf`) para gerar previsões sobre o conjunto de teste.  
Ou seja, o modelo utiliza as features de entrada (`X_test`) para estimar a classe (`label`) de cada amostra.

```python
y_test_pred = best_rf.predict(X_test)
```

#### 3.6.3 Criação e Salvamento do Arquivo de Submissão

Após gerar as previsões com o modelo, organizamos os resultados em um DataFrame contendo as colunas `id` e `labels` e salvamos em formato CSV, que pode ser enviado para submissão.

```python
# Criar DataFrame de submissão
df_submission = pd.DataFrame({
    'id': df_test['id'],
    'labels': y_test_pred
})

# Salvar em CSV
df_submission.to_csv('submission_rf_DECISIVO_CRISTIANO_ROMERO.csv', index=False)
print("Arquivo de submissão salvo como submission_rf.csv")
```


In [15]:
# Separar features do conjunto de teste
drop_cols_test = [col for col in ['id', 'situacao_funding', 'labels'] if col in df_test.columns]
X_test = df_test.drop(columns=drop_cols_test)

# Prever os rótulos do conjunto de teste
y_test_pred = best_rf.predict(X_test)

# Salvar para submissão
df_submission = pd.DataFrame({
    'id': df_test['id'],
    'labels': y_test_pred
})
df_submission.to_csv('submission_rf_DECISIVO_CRISTIANO_ROMERO.csv', index=False)
print("Arquivo de submissão salvo como submission_rf.csv")

Arquivo de submissão salvo como submission_rf.csv
